## 4 NSA und SQL (7 Punkte)
Copyright Jens Dittrich & Christian Schön, [Big Data Analytics Group](https://bigdata.uni-saarland.de/), [CC-BY-SA](https://creativecommons.org/licenses/by-sa/4.0/legalcode)

## Requirements

This notebook is based on the sqlite3-kernel by Andrew Brownan which is [available on GitHub](https://github.com/brownan/sqlite3-kernel). As the kernel is based on the bash shell, it will only run on Unix systems, but not on Windows.

The easiest way to use this notebook is to use our `Vagrantfile` as explained [here](https://github.com/BigDataAnalyticsGroup/vagrant-bde).

Alternatively, if you want to install the sqlite kernel yourself, follow these steps:
1. Download the repository as zip file or clone it using git, if necessary unpack the archive.
2. If you are using a virtual machine, copy the folder to a location accessible by the virtual machine, e.g. a shared folder. Run your virtual machine and if necessary start your python environment.
3. Move to the folder containing the kernel and execute the following commands:
  - python setup.py install
  - python -m sqlite3_kernel.install
  
If the kernel was successfully installed, you should now be able to start jupyter notebook and select "Sqlite3" as notebook type for new notebooks. 

In case of problems, you can delete the kernel using the following steps:
1. Look up the kernel name by executing the command: jupyter kernelspec list
2. Delete the kernel: jupyter kernelspec uninstall kernel_name

## Load Data

Before we can start analyzing the data, we first have to load data from the corresponding csv files into an appropriate database schema. This is fake data.

In [ ]:
PRAGMA foreign_keys = OFF;

DROP TABLE IF EXISTS purchases;
DROP TABLE IF EXISTS nutritionalValues;
DROP TABLE IF EXISTS livingIn;
DROP TABLE IF EXISTS households;
DROP TABLE IF EXISTS citizens;
DROP TABLE IF EXISTS articles;

PRAGMA foreign_keys = ON;

CREATE TABLE households (
    id INTEGER PRIMARY KEY,
    street TEXT,
    postcode INTEGER,
    city TEXT,
    floor INTEGER
);

CREATE TABLE citizens (
    id INTEGER PRIMARY KEY,
    firstname TEXT,
    lastname TEXT,
    birthday TEXT
);

CREATE TABLE livingIn (
    household_id INTEGER,
    citizen_id INTEGER,
    start TEXT,
    until TEXT,
    FOREIGN KEY(household_id) REFERENCES households(id),
    FOREIGN KEY(citizen_id) REFERENCES citizens(id),
    PRIMARY KEY(citizen_id, start, until)
);

CREATE TABLE articles (
    id INTEGER PRIMARY KEY,
    label TEXT,
    unit INTEGER
);

CREATE TABLE nutritionalValues (
    id INTEGER PRIMARY KEY,
    calories INTEGER,
    FOREIGN KEY(id) REFERENCES articles(id)
);

CREATE TABLE purchases (
    article_id INTEGER,
    citizen_id INTEGER,
    date TEXT,
    amount REAL,
    FOREIGN KEY(article_id) REFERENCES articles(id),
    FOREIGN KEY(citizen_id) REFERENCES citizens(id),
    PRIMARY KEY(article_id, citizen_id, date)
);

In [ ]:
-- enable csv mode:
.mode csv

-- import the necessary files:
.import data/nsa/households_no_header.csv households
.import data/nsa/citizens_no_header.csv citizens
.import data/nsa/livingIn_no_header.csv livingIn
.import data/nsa/articles_no_header.csv articles
.import data/nsa/nutritionalValues_no_header.csv nutritionalValues
.import data/nsa/purchases_no_header.csv purchases

-- enable pretty formatting:
.mode columns
.headers on

### Query 1

First we would like to identify the households that have a house number of 13 and in which at least 2 citizens live or lived that were born before 1920. Note that the house number is contained at the end of the `street` attribute of `households`. The output should contain the following attributes and should be ordered ascending by the street name:
1. The street of the household as "Straße",
2. The zip code of the household as "PLZ",
3. The city of the houshold as "Stadt".

In [ ]:
DROP VIEW IF EXISTS q1_student;
CREATE VIEW q1_student AS
-- insert your query here
-- ...

In [ ]:
-- TEST
-- Prepare the necessary table for result comparison and load the data
-- You need to execute this cell only once
-- Repeated execution will not affect test results, but lead to error messages as you try to import the same data multiple times

DROP TABLE IF EXISTS q1;
CREATE TABLE q1 (
    Straße TEXT,
    PLZ INTEGER,
    Stadt TEXT,
    PRIMARY KEY(Straße, PLZ, Stadt)
);

-- import query results
.mode csv
.import data/nsa/tests/q1_no_header.csv q1

In [ ]:
-- TEST
-- Note that this test compares the resulting tuples and does not ensure that your query is semantically correct.

-- compare query results
.mode columns

SELECT *
FROM (SELECT * FROM q1
      EXCEPT
      SELECT * FROM q1_student)
UNION
SELECT *
FROM (SELECT * FROM q1_student
      EXCEPT
      SELECT * FROM q1);
-- We expect an empty result.

### Query 2

Now, we would like to find for each article, which citizen(s) bought the most amount of it in a single purchase. Your output should contain the following attributes and should be ordered descending by the label of the article:

1. The label of the article as "Bezeichnung",
2. The maximum amount purchased in a single purchase of that particular article (either in liter or kilogram) as "Menge",
3. The first name of the citizen as "Vorname",
4. The last name of the citizen as "Nachname"

In [ ]:
DROP VIEW IF EXISTS q2_student;
CREATE VIEW q2_student AS
-- insert your query here
-- ...

In [ ]:
-- TEST
-- Prepare the necessary table for result comparison and load the data
-- You need to execute this cell only once
-- Repeated execution will not affect test results, but lead to error messages as you try to import the same data multiple times

DROP TABLE IF EXISTS q2;
CREATE TABLE q2 (
    Bezeichnung TEXT,
    Menge INTEGER,
    Vorname TEXT,
    Nachname TEXT,
    PRIMARY KEY(Bezeichnung)
);

-- import query results
.mode csv
.import data/nsa/tests/q2_no_header.csv q2

In [ ]:
-- TEST
-- Note that this test compares the resulting tuples and does not ensure that your query is semantically correct.

-- compare query results
.mode columns

SELECT *
FROM (SELECT q2.Bezeichnung, ROUND(q2.Menge, 5) AS Menge, q2.Vorname, q2.Nachname FROM q2
      EXCEPT
      SELECT q2_student.Bezeichnung, ROUND(q2_student.Menge, 5) AS Menge, q2_student.Vorname, q2_student.Nachname FROM q2_student)
UNION
SELECT *
FROM (SELECT q2_student.Bezeichnung, ROUND(q2_student.Menge, 5) AS Menge, q2_student.Vorname, q2_student.Nachname FROM q2_student
      EXCEPT
      SELECT q2.Bezeichnung, ROUND(q2.Menge, 5) AS Menge, q2.Vorname, q2.Nachname FROM q2);
-- We expect an empty result.

### Query 3

Lastly, we want to identify the citizens that moved in two households between 1900 and 1949 (both included). You should also compute the purchased calories for each of these citizens during the time period between 1900 and 1949. Note, that the `calories` in `nutritionalValues` are given in kcal/100g, while the `amount` of a bought article in `purchases` is given in kg. Therefore you need to convert kcal/100g in kcal/kg. The output should consist of the following attributes, ordered descending by "Kalorien".
1. The citizen's first name as "Vorname",
2. The citizen's last name as "Nachname",
3. The total amount of calories purchased during that time period by the citizen as "Kalorien".

In [ ]:
DROP VIEW IF EXISTS q3_student;
CREATE VIEW q3_student AS
-- insert your query here
-- ...

In [ ]:
-- TEST
-- Prepare the necessary table for result comparison and load the data
-- You need to execute this cell only once
-- Repeated execution will not affect test results, but lead to error messages as you try to import the same data multiple times

DROP TABLE IF EXISTS q3;
CREATE TABLE q3 (
    Vorname TEXT,
    Nachname TEXT,
    Kalorien REAL,
    PRIMARY KEY(Vorname, Nachname)
);

-- import query results
.mode csv
.import data/nsa/tests/q3_no_header.csv q3

In [ ]:
-- TEST
-- Note that this test compares the resulting tuples and does not ensure that your query is semantically correct.

-- compare query results
.mode columns

SELECT *
FROM (SELECT q3.Vorname, q3.Nachname, ROUND(q3.Kalorien, 5) AS Kalorien FROM q3
      EXCEPT
      SELECT q3_student.Vorname, q3_student.Nachname, ROUND(q3_student.Kalorien, 5) AS Kalorien FROM q3_student)
UNION
SELECT *
FROM (SELECT q3_student.Vorname, q3_student.Nachname, ROUND(q3_student.Kalorien, 5) AS Kalorien FROM q3_student
      EXCEPT
      SELECT q3.Vorname, q3.Nachname, ROUND(q3.Kalorien, 5) AS Kalorien FROM q3);
-- We expect an empty result.